In [1]:
from selenium import webdriver
from time import sleep, time
from pprint import pprint
import json
import re

In [45]:
driver = webdriver.Firefox()

In [89]:
url = 'https://www.coursera.org/browse/personal-development'

In [90]:
driver.get(url=url)

In [91]:
btn = driver.find_element_by_class_name('Button_clbp6a-o_O-icon_1ma9ny8-o_O-sm_pd780u')
btn.click()

In [92]:
check = driver.find_elements_by_class_name('input_3xha1c-o_O-active_ro0g1e')
cat_count = len(check)
cat_count

89

In [93]:
doc_list = []
s = 5
for i in range(cat_count):
    btn.click()
    cat = driver.find_elements_by_class_name('flex1_mv0w51')
    cat_name = cat[i].text
    check = driver.find_elements_by_class_name('input_3xha1c-o_O-active_ro0g1e')
    check[i].click()
    apply = driver.find_element_by_class_name('Button_clbp6a-o_O-primary_cv02ee-o_O-md_1jvotax')
    apply.click()
    sleep(s)
    # print("Category: {}".format(cat_name))
    
    unique_list = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    s_offset = 0
    e_offset = 1000
    while True:
        sleep(3)
        items = driver.find_elements_by_class_name('offering-wrapper')
        for item in items:
            if item not in unique_list:
                unique_list.append(item)
                
                d = {}
                info = item.find_element_by_class_name('product-info')
                prod = info.find_element_by_class_name('product-name')
                part = info.find_element_by_class_name('partner-name')
                course_type = info.find_element_by_css_selector('div')
                ratings = item.find_element_by_class_name('ratings-text')
                link = item.find_element_by_class_name('search-offering-card')
                meta_data = item.find_elements_by_class_name('product-metadata')
                # print(prod.text, end='\t')
                # print(part.text, end='\t')
                # print(ratings.text)
                
                d['category'] = cat_name
                d['title'] = prod.text
                d['partner'] = part.text
                d['link'] = link.get_attribute(name='href')
                d['rating'] = float(ratings.text)
                d['type'] = course_type.text
                try:
                    d['course_count'] = meta_data[0].text
                except:
                    d['course_count'] = '1 COURSE'
                try:
                    d['duration'] = meta_data[1].text
                except:
                    d['duration'] = '0 HOURS'
                try:
                    d['level'] = meta_data[2].text
                except:
                    d['level'] = 'BEGINNER'
                try:
                    rating_c = item.find_element_by_class_name('ratings-count')
                    d['users'] = int(rating_c.text.replace('(','').replace(')','').replace(',',''))
                except Exception as e:
                    d['users'] = 0
                doc_list.append(d)
                
        driver.execute_script('window.scrollTo('+str(s_offset)+', ' + str(e_offset)+');')
        new_height = driver.execute_script("return window.scrollY")
        s_offset = e_offset
        e_offset += 1000
        if new_height == last_height:
            break
        last_height = new_height
    
    btn.click()
    check = driver.find_elements_by_class_name('input_3xha1c-o_O-active_ro0g1e')
    check[0].click()
    apply = driver.find_element_by_class_name('Button_clbp6a-o_O-primary_cv02ee-o_O-md_1jvotax')
    apply.click()
    print('\rProgress: {}/{}'.format(i+1, cat_count), end='')
    # print()
    sleep(3)


Progress: 89/89

In [94]:
for doc in doc_list:
    doc['category'] = re.sub(' \([0-9]+\)', '', doc['category'])

In [95]:
with open(url.split('/')[-1] + ' ' + str(time()) + '.txt', 'w', encoding='utf-16') as f:
    pprint(doc_list, stream=f)

In [96]:
with open(url.split('/')[-1] + ' ' + str(time()) + '.json', 'w') as f:
    f.write(json.dumps(doc_list, indent=4))

In [86]:
processed_list = []
titles = []
for d in doc_list:
    if d['title'] in titles:
        for i in range(len(processed_list)):
            if processed_list[i]['title'] == d['title']:
                processed_list[i]['category'].append(d['category'])
                break
        continue
    d['category'] = [d['category']]
    d['course_count'] = int(d['course_count'].replace(' COURSE', '').replace('S', ''))
    d['duration'] = int(re.sub('[ ]*HOUR[S]?', '', d['duration']))
    processed_list.append(d)
    titles.append(d['title'])

In [98]:
with open(url.split('/')[-1] + ' ' + str(time()) + '.txt','w', encoding='utf-16') as f:
    pprint(processed_list, stream=f)

In [99]:
with open(url.split('/')[-1] + ' ' + str(time()) + '.json', 'w') as f:
    f.write(json.dumps(processed_list, indent=4))

In [100]:
len(doc_list)

143

In [101]:
len(processed_list)

63